# Alignment with Numba

This notebook contains numba accelerated implementations of NSDTW, WSDTW, and SSDTW, along with code to conduct runtime experiments on these implementations. It also contains multithreaded and further parallelized implementations of WSDTW.

In [ ]:
import numpy as np
import scipy.spatial.distance as dist
import matplotlib.pyplot as plt
import librosa as lb
import os.path
from pathlib import Path
import pickle
import multiprocessing
import time
import gc
from numba import jit, njit, float64, uint32, boolean
from fastdtw import fastdtw
from scipy.spatial.distance import cosine

In [ ]:
##### Change this cell to suit your file structure #####
OUT_ROOT = Path().absolute() # Output root directory (this is where features, paths, etc. will be saved)
########################################################

In [ ]:
FEATURES_ROOT = OUT_ROOT / 'features'

## DTW and Subsequence DTW

In [ ]:
DTWDefaultSteps = np.array([[1, 1, 2],
                            [1, 2, 1]], dtype = np.uint32)

DTWDefaultWeights = np.array([2, 3, 3], dtype = np.float64)


subseqDTWDefaultSteps = np.array([[1, 1, 2],
                                  [1, 2, 1]], dtype = np.uint32)

subseqDTWDefaultWeights = np.array([1, 1, 2], dtype = np.float64)


MAX_FLOAT = float('inf')

In [ ]:
@njit
def DTW_Cost_To_DandB(C, Steps = DTWDefaultSteps, weights = DTWDefaultWeights, subsequence=False):
    '''
    Find the accumulated cost matrix and backtrace matrix from a cost matrix using Dynamic time warping
    
    Arguments:
    C -- The Cost matrix
    Steps -- The available steps, where the first row is the row steps, the second row is the column steps
    weights -- The weights of the steps
    subsequence -- True if subsequence DTW should be performed rather than standard DTW
             
    Returns:
    D -- The accumulated cost matrix
    B -- The backtrace matrix
    '''
    '''
    Section for verifying input
    '''
    # Separate steps
    rowSteps = Steps[0,:]
    colSteps = Steps[1,:]

    # Define Relevant Variables
    numRows = C.shape[0]
    numCols = C.shape[1]
    
    numDifSteps = len(weights)
    maxRowStep = max(rowSteps)
    maxColStep = max(colSteps)
    
    # Set up accumulated cost matrix D and backtrace matrix B
    D = np.ones((numRows + maxRowStep, numCols + maxColStep), dtype = np.float64) * MAX_FLOAT
    B = np.zeros((numRows, numCols), dtype = np.uint32)
    
    # Fill up D and B
    if subsequence:  # Initialize entire bottom row of D for subsequence
        D[maxRowStep, maxColStep:] = C[0,:]
    else:
        D[maxRowStep, maxColStep] = C[0,0]  # Initialize bottom corner if for standard DTW
    for row in range(maxRowStep, numRows + maxRowStep, 1):
        for col in range(maxColStep, numCols + maxColStep, 1):
            bestCost = D[row, col]
            bestCostIndex = 0
            # Go through each step, find the best one
            for stepIndex in range(numDifSteps):
                costForStep = D[row - rowSteps[stepIndex], col - colSteps[stepIndex]] + weights[stepIndex] * C[row - maxRowStep, col - maxColStep]
                if costForStep < bestCost:
                    bestCost = costForStep
                    bestCostIndex = stepIndex
            # Save best cost and step
            D[row,col] = bestCost
            B[row - maxRowStep, col - maxColStep] = bestCostIndex
    # Return accumulated cost matrix D and backtrace matrix B
    return D[maxRowStep:, maxColStep:], B

In [ ]:
@njit
def DTW_Backtrace(D, B, Steps=DTWDefaultSteps, subsequence=False, startCol=-1):
    '''
    Backtrace through an accumulated cost matrix and backtrace matrix to find a path
    
    Arguments:
    D -- The accumulated cost matrix
    B -- The backtrace matrix
    Steps -- The available steps
    subsequence -- True if subsequence DTW should be performed rather than standard DTW
    startCol -- The column to begin backtracing from, or -1 if not specified
    
    Returns:
    fwdPath -- A 2d numpy array storing the optimal path. The first row is the path through the rows.
            The second row is the path through the columns
    '''
    '''
    Section for verifying input
    '''
    # Separate steps
    rowSteps = Steps[0,:]
    colSteps = Steps[1,:]
    
    # Initialize variables
    numRows = D.shape[0]
    numCols = D.shape[1]
    
    curRow = numRows - 1  # Always start at last row
    curCol = numCols - 1  # Standard DTW: Start at top-right corner
    if startCol >= 0:
        curCol = startCol
    elif subsequence:  # Subsequence: Choose lowest cost of top row
        curCol = np.argmin(D[numRows-1,:])
    
    endCol = curCol
    endCost = D[curRow, curCol]
    stepsInPath = 1
    stepIndex = 0
    done = (subsequence and curRow == 0) or (curRow == 0 and curCol == 0)
    path = np.zeros((2, numRows + numCols), dtype=np.uint32) # make as large as could need, then chop at the end
    path[0, 0] = curRow
    path[1, 0] = curCol
    
    # Backtrace
    while not done:
        
        if D[curRow, curCol] == MAX_FLOAT:  # No path exists to current location
            break
        
        # you're done if you've made it to the bottom left (non sub-sequence)
        # or just the bottom (sub-sequence)
        # find the step size
        curStepIndex = B[curRow, curCol]
        curRowStep = rowSteps[curStepIndex]
        curColStep = colSteps[curStepIndex]
        # backtrack by 1 step
        curRow = curRow - curRowStep
        curCol = curCol - curColStep
        # add your new location onto the path
        path[0, stepsInPath] = curRow
        path[1, stepsInPath] = curCol
        stepsInPath = stepsInPath + 1
        # check to see if you're done
        done = (subsequence and curRow == 0) or (curRow == 0 and curCol == 0)
        
    # reverse the path (a matrix with two rows) and return it
    fwdPath = np.fliplr(path[:, 0:stepsInPath])
    return fwdPath

In [ ]:
# Compile this function in object mode to allow for np.load and time profiling
@jit(forceobj = True)
def DTW(queryFeatureFile, refFeatureFile, Steps = DTWDefaultSteps, weights = DTWDefaultWeights, subsequence = False, \
        outfile = None, profile = False):
    '''
    Run DTW on query and reference feature matrices
    
    Arguments:
    queryFeatureFile -- The file containing the query feature matrix
    refFeatureFile -- The file containing the reference feature matrix
    Steps -- The steps matrix. The first row is the steps in along the rows,
             the second row is the steps along the columns
    weights -- The weights for the steps
    subsequence -- True if subsequence DTW should be performed rather than standard DTW
    
    Returns:
    path -- A 2d numpy array storing the optimal path. The first row is the path through the rows.
            The second row is the path through the columns
    '''
    # Extract Feature matrices
    F1 = np.load(queryFeatureFile, allow_pickle = True)
    F2 = np.load(refFeatureFile, allow_pickle = True)
    
    # empty file if no valid path possible
    if not subsequence and max(F1.shape[1], F2.shape[1]) / min(F1.shape[1], F2.shape[1]) >= 2:
        if outfile:
            pickle.dump(None, open(outfile, 'wb'))
        return None
    
    # start time logging
    times = []
    times.append(time.time())
    
    # Compute Cost Matrix
    C = 1 - F1.T @ F2 # cos distance metric
    
    times.append(time.time())
    
    # Get D and B
    D, B = DTW_Cost_To_DandB(C, Steps = Steps, weights = weights, subsequence=subsequence)
    
    times.append(time.time())
    
    # Backtrace and return
    wp = DTW_Backtrace(D, B, Steps=Steps, subsequence=subsequence)
    
    times.append(time.time())
    
    # logging result if outfile exists
    if outfile:
        pickle.dump(wp, open(outfile, 'wb'))
    
    # return extra timing array if profiling timing
    if profile:
        return wp, np.diff(times)
    else:
        return wp

In [ ]:
def DTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, subsequence):
    
    outdir.mkdir(parents=True, exist_ok=True)
    
    # prep inputs for parallelization
    inputs = []
    with open(querylist, 'r') as f:
        for line in f:
            parts = line.strip().split(' ')
            assert len(parts) == 2
            featfile1 = (featdir1 / parts[0]).with_suffix('.npy')
            featfile2 = (featdir2 / parts[1]).with_suffix('.npy')
            queryid = os.path.basename(parts[0]) + '__' + os.path.basename(parts[1])
            outfile = (outdir / queryid).with_suffix('.pkl')
            if os.path.exists(outfile):
                print(f"Skipping {outfile}")
            else:
                inputs.append((featfile1, featfile2, steps, weights, subsequence, outfile))

    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(DTW, inputs)
    
    return

In [ ]:
query_list = 'cfg_files/query.test.list'
featdir1 = FEATURES_ROOT / 'clean'
featdir2 = FEATURES_ROOT / 'clean' # in case you want to align clean vs noisy
outdir = OUT_ROOT / 'experiments_test/clean/DTW'
n_cores = 1
steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
weights = np.array([2,3,3])
subseq = False
DTW_batch(query_list, featdir1, featdir2, outdir, n_cores, steps, weights, subseq)

## Non-ordered Segmental DTW

In [ ]:
# Compile this function in object mode to allow for np.load and time profiling
@jit(forceobj = True)
def NSDTW(queryFeatureFile, refFeatureFile, segments, Steps = subseqDTWDefaultSteps, weights = subseqDTWDefaultWeights, \
          outfile = None, profile = False):
    '''
    Runs a non-ordered segmental DTW between query and reference features matrices
    
    Arguments:
    queryFeatureFile -- The file containing the query feature matrix
    refFeatureFile -- The file containing the reference feature matrix
    segments -- The number of segments to divide F1 into
    Steps -- The allowed steps
    weights -- The weights for the steps
    
    Returns:
    path -- The optimal non-ordered, segmented alignment path between F1 and F2
    '''
    # Extract Feature matrices
    F1 = np.load(queryFeatureFile, allow_pickle = True)
    F2 = np.load(refFeatureFile, allow_pickle = True)

    if max(F1.shape[1], F2.shape[1]) / min(F1.shape[1], F2.shape[1]) >= 2: # no valid path possible
        if outfile:
            pickle.dump(None, open(outfile, 'wb'))
        return None
    
    # start time logging
    times = []
    times.append(time.time())
    
    # Compute Cost
    C = 1 - F1.T @ F2
    
    times.append(time.time())
    
    # Subsequence DTW each segment without backtracing
    segLength = int(np.ceil(F1.shape[1] / segments))
    Cseg = np.zeros((segments + 1, C.shape[1]), dtype = np.float64)
    Dparts = []
    Bparts = []
    for i in range(segments):
        segStart = i * segLength
        segEnd = min(segStart + segLength, C.shape[0])
        # Ensuring that the segment is contiguous here ensures best performance in later computations
        currentSeg = np.ascontiguousarray(C[segStart:segEnd,:])
        D_i, B_i = DTW_Cost_To_DandB(currentSeg, Steps = Steps, weights = weights, subsequence = True)
        
        # Store D_i and B_i for segment
        Dparts.append(D_i)
        Bparts.append(B_i)
    
    times.append(time.time())
    
    # Backtrace through segments with segment level path as guide
    # Initialize path: make as large as could need, then chop at the end
    path = []
    stepsInPath = 0
    # Frame level backtrace segment by segment
    for i in range(segments):
        pathSeg = DTW_Backtrace(Dparts[i], Bparts[i], Steps = Steps, subsequence = True)
        # Add offset to row indices so they match with overall path
        pathSeg[0,:] = pathSeg[0,:] + (i * segLength)

        # Append fragment to full path
        path.append(pathSeg.copy())
    
    wp_merged = np.hstack(path)
    
    times.append(time.time())
    
    if outfile:
        pickle.dump(wp_merged, open(outfile, 'wb'))

    if profile:
        return wp_merged, np.diff(times)
    else:
        return wp_merged

In [ ]:
def SDTW_batch(querylist, featdir1, featdir2, outdir, n_cores, steps, weights, numSegments, fn):

    outdir.mkdir(parents=True, exist_ok=True)
    
    # prep inputs for parallelization
    inputs = []
    with open(querylist, 'r') as f:
        for line in f:
            parts = line.strip().split(' ')
            assert len(parts) == 2
            featfile1 = (featdir1 / parts[0]).with_suffix('.npy')
            featfile2 = (featdir2 / parts[1]).with_suffix('.npy')
            queryid = os.path.basename(parts[0]) + '__' + os.path.basename(parts[1])
            outfile = (outdir / queryid).with_suffix('.pkl')
            if os.path.exists(outfile):
                print(f"Skipping {outfile}")
            else:
                inputs.append((featfile1, featfile2, numSegments, steps, weights, outfile))

    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(fn, inputs)

    return

In [ ]:
query_list = 'cfg_files/query.test.list'
featdir1 = FEATURES_ROOT / 'clean'
featdir2 = FEATURES_ROOT / 'clean' # in case you want to align clean vs noisy

n_cores = 1
steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
weights = np.array([1,1,2])
segmentVals = [2, 4, 8, 16, 32] 
for numSegments in segmentVals:
    outdir = OUT_ROOT / f'experiments_test/clean/NSDTW_{numSegments}'
    SDTW_batch(query_list, featdir1, featdir2, outdir, n_cores, steps, weights, numSegments, NSDTW)

## Weakly-ordered Segmental DTW

In [ ]:
@njit
def getSegmentEndingLocs(wp):
    '''
    Takes in a segment level path through and returns the columns where each segment ends
    
    Arguments:
    wp -- A segment level path (as a 2d numpy array with 2 rows)
    
    Returns:
    endLocs -- A list where the ith entry is the column where the ith segment ends
    '''
    prevLoc = wp[:,0] # [r,c]
    endLocs = []
    for i in range(wp.shape[1]):
        curLoc = wp[:,i]
        if curLoc[0] != prevLoc[0]: # if row changes
            endLocs.append(curLoc[1])
        prevLoc = curLoc
        
    return endLocs

In [ ]:
# Compile this function in object mode to allow for np.load and time profiling
@jit(forceobj = True)
def WSDTW(queryFeatureFile, refFeatureFile, segments, Steps = subseqDTWDefaultSteps, weights = subseqDTWDefaultWeights, \
          outfile = None, profile = False):
    '''
    Runs a weakly-ordered segmental DTW between query and reference features matrices
    
    Arguments:
    queryFeatureFile -- The file containing the query feature matrix
    refFeatureFile -- The file containing the reference feature matrix
    segments -- The number of segments to divide F1 into
    Steps -- The allowed steps
    weights -- The weights for the steps
    
    Returns:
    path -- The optimal weakly-ordered, segmented alignment path between F1 and F2
    '''
    # Extract Feature matrices
    F1 = np.load(queryFeatureFile, allow_pickle = True)
    F2 = np.load(refFeatureFile, allow_pickle = True)

    if max(F1.shape[1], F2.shape[1]) / min(F1.shape[1], F2.shape[1]) >= 2: # no valid path possible
        if outfile:
            pickle.dump(None, open(outfile, 'wb'))
        return None
    
    # start time logging
    times = []
    times.append(time.time())
    
    # Compute Cost
    C = 1 - F1.T @ F2
    
    times.append(time.time())
    
    # Subsequence DTW each segment without backtracing
    segLength = int(np.ceil(F1.shape[1] / segments))
    Cseg = np.zeros((segments + 1, C.shape[1]), dtype = np.float64)
    Dparts = []
    Bparts = []
    for i in range(segments):
        segStart = i * segLength
        segEnd = min(segStart + segLength, C.shape[0])
        # Ensuring that the segment is contiguous here ensures best performance in later computations
        currentSeg = np.ascontiguousarray(C[segStart:segEnd,:])
        D_i, B_i = DTW_Cost_To_DandB(currentSeg, Steps = Steps, weights = weights, subsequence = True)
        
        # Store D_i and B_i for segment and construct Cseg
        Dparts.append(D_i)
        Bparts.append(B_i)
        
        Cseg[i + 1,:] = D_i[-1,:]
    
    times.append(time.time())
    
    # run segment-level DTW (Not subsequence)
    segmentSteps = np.array([[0, 1],
                             [1, C.shape[0]//(2 * segments)]],
                            dtype=np.uint32)
    segmentWeights = np.array([0, 1])
    
    Dseg, Bseg = DTW_Cost_To_DandB(Cseg, Steps = segmentSteps, weights = segmentWeights)
    
    times.append(time.time())
    
    wpseg = DTW_Backtrace(Dseg, Bseg, Steps=segmentSteps)
    
    times.append(time.time())
    
    # Backtrace through segments with segment level path as guide
    path = []
    # Frame level backtrace segment by segment
    segmentEndIdxs = getSegmentEndingLocs(wpseg)
        
    for i, endidx in enumerate(segmentEndIdxs):
        pathSeg = DTW_Backtrace(Dparts[i], Bparts[i], Steps = Steps, subsequence = True, startCol = endidx)
        # Add offset to row indices so they match with overall path
        pathSeg[0,:] = pathSeg[0,:] + (i * segLength)

        # Append fragment to full path
        path.append(pathSeg.copy())
    
    wp_merged = np.hstack(path)
    
    times.append(time.time())
    
    if outfile:
        pickle.dump(wp_merged, open(outfile, 'wb'))

    if profile:
        return wp_merged, np.diff(times)
    else:
        return wp_merged

In [ ]:
query_list = 'cfg_files/query.test.list'
featdir1 = FEATURES_ROOT / 'clean'
featdir2 = FEATURES_ROOT / 'clean' # in case you want to align clean vs noisy
n_cores = 1
steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
weights = np.array([1,1,2])
segmentVals = [2, 4, 8, 16, 32] 
for numSegments in segmentVals:
    outdir = OUT_ROOT / f'experiments_test/clean/WSDTW_{numSegments}'
    SDTW_batch(query_list, featdir1, featdir2, outdir, n_cores, steps, weights, numSegments, WSDTW)

## Multithreded WSDTW

In [ ]:
def WSDTW_multi(F1, F2, K, costMetric = "cos", profile = False):
    '''
    Runs a multi-threaded, parallelized weakly-ordered segmental DTW between two feature matrices
    
    Arguments:
    F1 -- The query feature matrix
    F2 -- The reference feature matrix
    K -- The number of segments to divide F1 into
    costMetric -- The cost metric to use to compute the cost matrix
    
    Returns:
    path -- The optimal weakly-ordered, segmented alignment path between F1 and F2
    '''
    if max(F1.shape[1], F2.shape[1]) / min(F1.shape[1], F2.shape[1]) >= 2: # no valid path possible
        if outfile:
            pickle.dump(None, open(outfile, 'wb'))
        return None
    
    # Set up variables
    frameSteps = np.array([[1, 1, 2],
                           [1, 2, 1]], dtype = np.uint32)

    frameWeights = np.array([1, 1, 2], dtype = np.float64)
    segLength = int(np.ceil(F1.shape[1] / K))
    
    times = []
    times.append(time.time())
    # DTW each segment in parallel
    # Set up task list
    tasks = [(F1, F2, costMetric, segLength, i) for i in range(K)]
    
    # Set up pool
    with multiprocessing.Pool(processes = K) as p:
        times.append(time.time())
         # Perform subsequence DTW on each segment in parallel
        segmentResults = p.starmap(DTWSegment, tasks)
        
    times.append(time.time())

    # Run segment-level DTW (Not subsequence)
    segmentSteps = np.array([[0, 1],
                             [1, F1.shape[1]//(2 * K)]],
                            dtype=np.uint32)
    segmentWeights = np.array([0, 1])
    
    Cseg = np.zeros((K + 1, F2.shape[1]), dtype = np.float64)
    for i, (D_i, _) in enumerate(segmentResults):
        Cseg[i+1,:] = D_i[-1,:]

    times.append(time.time())

    Dseg, Bseg = DTW_Cost_To_DandB(Cseg, Steps = segmentSteps, weights = segmentWeights)
    
    times.append(time.time())
    
    wpseg = DTW_Backtrace(Dseg, Bseg, Steps=segmentSteps)
    
    times.append(time.time())

    # Backtrace through segments with segment level path as guide
    # Set up task list (use segmentResults)
    segmentEndIdxs = getSegmentEndingLocs(wpseg)
    backtraceTasks = [(D, B, segmentEndIdxs[i], segLength, i) for i, (D, B) in enumerate(segmentResults)]
    
    # Perform backtrace in parallel
    with multiprocessing.Pool(processes = K) as p:
        pathSegments = p.starmap(backtraceSegment, backtraceTasks)
    times.append(time.time())
    
    # Compile path and return
    wp_merged = np.concatenate(pathSegments, axis = 1)
    if profile:
        return wp_merged, np.diff(times)
    else:
        return wp_merged

In [ ]:
@jit(forceobj = True)
def DTWSegment(F1, F2, costMetric, segLength, segNum):
    '''
    Extracts a single segment of F1 and performs subsequence DTW between it and F2
    
    Arguments:
    F1 -- The query feature matrix
    F2 -- The reference feature matrix
    costMetric -- The cost metric to use to compute the cost matrix
    segLength -- The length of each segment of the query feature matrix
    segNum -- The index of the segment to use (0 to K-1, inclusive)
    
    Returns:
    D_i -- The accumulated cost matrix for the relevant segment
    B_i -- The backtrace matrix for the relevant segment
    '''
    # Get segment
    seg = extractSegment(F1, segLength, segNum)
    
    
    # Compute costs:
    C = dist.cdist(seg.T, F2.T, metric = costMetric)  # scipy's cdist wants dimensions reversed
    
    # DTW
    Steps = np.array([[1, 1, 2],
                      [1, 2, 1]], dtype = np.uint32)

    weights = np.array([1, 1, 2], dtype = np.float64)
    
    return DTW_Cost_To_DandB(C, Steps = Steps, weights = weights, subsequence = True)

In [ ]:
@njit
def extractSegment(F1, segLength, segNum):
    '''
    Extracts a single segment of F1
    
    Arguments:
    F1 -- The query feature matrix
    segLength -- The length of each segment of the query feature matrix
    segNum -- The index of the segment to use (0 to K-1, inclusive)
    
    Returns:
    segment -- A single segment of F1
    '''
    segStart = segNum * segLength
    segEnd = min(segStart + segLength, F1.shape[1])
    # Ensuring that the segment is contiguous here ensures best performance in later computations
    return np.ascontiguousarray(F1[:,segStart:segEnd])

In [ ]:
@njit
def backtraceSegment(D, B, startCol, segLength, segNum):
    '''
    Backtrace through a segment and return the path with the row indices offset so they match the overall path
    
    Arguments:
    D -- The cumulative cost matrix for the segment
    B -- The backtrace matrix for the segment
    startCol -- The column to start backtracing from
    segLength -- The length of each segment of the query feature matrix
    segNum -- The index of the segment to use (0 to K-1, inclusive)
    
    Returns:
    path -- The path through the segment with the row indices offset so they match the overall path
    '''
    # Set up variables
    Steps = np.array([[1, 1, 2],
                      [1, 2, 1]], dtype = np.uint32)
    
    # Backtrace
    path = DTW_Backtrace(D, B, Steps = Steps, subsequence = True, startCol = startCol)
    
    # Add offset to row indices so they match with overall path
    path[0,:] = path[0,:] + (segNum * segLength)
    return path

## Further Parallelized WSDTW

In [ ]:
def moreParallelWSDTW(F1, F2, K, M):
    '''
    Runs a more parallelized version of WSDTW. Each subsequence alignment is parallelized.
    
    Arguments:
    F1 -- The query feature matrix
    F2 -- The reference feature matrix
    K -- The number of fragments
    M -- The number of sub-fragments
    
    Returns:
    wp_merged -- The WSDTW path
    '''
    Steps = np.array([[1, 1, 2],
                      [1, 2, 1]], dtype = np.uint32)

    weights = np.array([1, 1, 2], dtype = np.float64)
    
    if max(F1.shape[1], F2.shape[1]) / min(F1.shape[1], F2.shape[1]) >= 2: # no valid path possible
        if outfile:
            pickle.dump(None, open(outfile, 'wb'))
        return None
    
    # Compute Cost
    C = 1 - F1.T @ F2
    
    # Subsequence DTW each segment without backtracing
    segLength = int(np.ceil(F1.shape[1] / K))
    Cseg = np.zeros((K + 1, C.shape[1]), dtype = np.float64)
    Dparts = []
    Bparts = []
    for i in range(K):
        segStart = i * segLength
        segEnd = min(segStart + segLength, C.shape[0])
        # Ensuring that the segment is contiguous here ensures best performance in later computations
        currentSeg = np.ascontiguousarray(C[segStart:segEnd,:])
        # Further divide each segment into fragments
        D_i = np.zeros(currentSeg.shape)
        B_i = np.ones(currentSeg.shape)
        fragLength = int(np.ceil(currentSeg.shape[1] / M) + 2 * currentSeg.shape[0])
        for j in range(M):
            fragStart = j * fragLength - j * 2 * currentSeg.shape[0]
            fragEnd = min(fragStart + fragLength, currentSeg.shape[1])
            currentFrag = currentSeg[:, fragStart:fragEnd]
            
            D_frag, B_frag = DTW_Cost_To_DandB(currentFrag, Steps = Steps, weights = weights, subsequence = True)
            
            if fragStart == 0: # Use all of first fragment
                D_i[:, 0:fragEnd] = D_frag
                B_i[:, 0:fragEnd] = B_frag
            else: # Discard overlap for later fragments
                D_i[:, fragStart + 2 * currentSeg.shape[0]:fragEnd] = D_frag[:, 2 * currentSeg.shape[0]:]
                B_i[:, fragStart + 2 * currentSeg.shape[0]:fragEnd] = B_frag[:, 2 * currentSeg.shape[0]:]        
        # Store D_i and B_i for segment and construct Cseg
        Dparts.append(D_i)
        Bparts.append(B_i)
        
        Cseg[i + 1,:] = D_i[-1,:]

    # run segment-level DTW (Not subsequence)
    segmentSteps = np.array([[0, 1],
                             [1, C.shape[0]//(2 * K)]],
                            dtype=np.uint32)
    segmentWeights = np.array([0, 1])
    
    Dseg, Bseg = DTW_Cost_To_DandB(Cseg, Steps = segmentSteps, weights = segmentWeights)
    wpseg = DTW_Backtrace(Dseg, Bseg, Steps=segmentSteps)
    
    # Backtrace through segments with segment level path as guide
    path = []
    # Frame level backtrace segment by segment
    segmentEndIdxs = getSegmentEndingLocs(wpseg)
    for i, endidx in enumerate(segmentEndIdxs):
        pathSeg = DTW_Backtrace(Dparts[i], Bparts[i], Steps = Steps, subsequence = True, startCol = endidx)
        # Add offset to row indices so they match with overall path
        pathSeg[0,:] = pathSeg[0,:] + (i * segLength)

        # Append fragment to full path
        path.append(pathSeg.copy())
    
    wp_merged = np.hstack(path)

    return wp_merged

## Strongly-ordered Segmental DTW

In [ ]:
@njit
def SSDTW_Segment_Level_DTW(Cseg, Tseg):
    '''
    Performs the segment level DTW for strongly-ordered segmental DTW
    Steps not required because they are variable based on Tseg
    
    Arguments:
    Cseg -- The segment level cost matrix
    Tseg -- A matrix where the i,jth entry is the column of the optimal path through segment i ending at column j.
            This is used to determine the possible segment level steps that ensure strong ordering
            
    Returns:
    Dseg -- The segment level accumulated cost matrix
    Bseg -- The segment level backtrace matrix. Since steps are variable, the i, jth entry of the backtrace matrix
            stores either -1 for a (1, 0) "skip" step or the colStep value of a (1, colStep) step ending at i,j
    '''

    # Define Relevant Variables
    numRows = Cseg.shape[0]
    numCols = Cseg.shape[1]
    
    # Set up accumulated cost matrix D and backtrace matrix B
    Dseg = np.ones((numRows + 1, numCols), dtype = np.float64) * MAX_FLOAT
    Dseg[0,:] = 0
    
    # (0,1) transition by default
    Bseg = np.zeros((numRows+1, numCols), dtype = np.int32) - 1
    
    # Fill up D and B
    for row in range(1, numRows+1):
        for col in range(numCols):
            # (0,1) transition is skip with weight 0
            if col==0:
                skipCost = MAX_FLOAT
            else:
                skipCost = Dseg[row, col-1]
            Dseg[row, col] = skipCost
            
            traverseStartCol = Tseg[row-1,col]
            if traverseStartCol >= 0: # is it possible to traverse previous segment and end up here?
                # Traverse segment with weight 1
                traverseSegCost = Dseg[row-1, traverseStartCol] + Cseg[row-1, col]
                # If traversing here is best option, store the column where the path starts
                if traverseSegCost < skipCost:
                    Dseg[row, col] = traverseSegCost
                    # TraverseStartCol represents where to start backtracing in the next row
                    Bseg[row, col] = traverseStartCol
    
    return Dseg, Bseg

In [ ]:
@njit
def SSDTW_Segment_Level_Backtrace(Dseg, Bseg):
    '''
    Backtraces through segments enforcing a strongly-ordered path
    
    Arguments:
    Dseg -- The segmental accumulated cost matrix
    Bseg -- The segmental backtrace matrix. Stores -1 for (0,1) transitions and colStep for (1, colStep) transitions
    
    Returns:
    path -- A vector where the ith element is the column where the ith segment should end
    '''
    # Initialize variables
    numRows = Dseg.shape[0]
    numCols = Dseg.shape[1]
    curRow = numRows - 1
    curCol = numCols - 1
    # Path will have one entry for every row except for the bottom one
    path = np.zeros(numRows-1, dtype = np.uint32)
    stepsInPath = 0
        
    # Backtrace until reaching bottom row
    while curRow > 0:
        
        if Dseg[curRow, curCol] == MAX_FLOAT:
            print('A path is not possible')
            break
        
        # (1,0) "skip" transitions (-1 in Bseg) do not get added to path
        if Bseg[curRow, curCol] < 0:
            curCol = curCol - 1
        # For segment traversals, store segment end location in the path.
        else:
            path[stepsInPath] = curCol
            # Continue backtracing where the segment tranversal began
            curCol = Bseg[curRow, curCol]
            curRow = curRow - 1
            stepsInPath = stepsInPath + 1
    # Remember to flip the backtraced path
    return path[::-1]

In [ ]:
@njit
def calcTseg(D_i, B_i, Steps = subseqDTWDefaultSteps):
    '''
    Calculate a row of Tseg for SSDTW from the the accumulated cost and backtrace matrix for that segment
    
    Arguments:
    D_i -- The accumulated cost matrix for the ith segment
    B_i -- The backtrace matrix for the ith segment
    Steps -- The available steps
    
    Returns:
    Tseg_i -- The ith row of Tseg
    '''
    # Initializing variables
    numRows = D_i.shape[0]
    numCols = D_i.shape[1]
    rowSteps = Steps[0,:]
    colSteps = Steps[1,:]
    # Default to -1, meaning no path ending at that location is possible
    Tseg_i = np.zeros(numCols, dtype = np.uint32) - 1
    
    # Fill in Tseg_i by backtracing from each column
    # backtrace from every location
    for endCol in range(numCols):
        curCol = endCol
        curRow = numRows - 1
        while curRow > 0:
            if D_i[curRow, curCol] == MAX_FLOAT: # no valid path
                Tseg_i[curCol] = -1
                break

            curStepIndex = B_i[curRow, curCol]
            curRow = curRow - rowSteps[curStepIndex]
            curCol = curCol - colSteps[curStepIndex]
            if curRow == 0:
                Tseg_i[endCol] = curCol
    
    return Tseg_i

In [ ]:
# Compile this function in object mode to allow for np.load and time profiling
@jit(forceobj = True)
def SSDTW(queryFeatureFile, refFeatureFile, segments, Steps = subseqDTWDefaultSteps, weights = subseqDTWDefaultWeights, \
          outfile=None, profile=False):
    '''
    Aligns query and reference with strictly-ordered segmental DTW
    
    Arguments:
    queryFeatureFile -- The file containing the query feature matrix
    refFeatureFile -- The file containing the reference feature matrix
    segments -- The number of segments
    Steps -- The allowable steps
    weights -- The weights of the steps
    
    Returns:
    path -- The optimal strictly ordered segmental path
    '''
    # Extract Feature matrices
    F1 = np.load(queryFeatureFile, allow_pickle = True)
    F2 = np.load(refFeatureFile, allow_pickle = True)
    

    swap = (F1.shape[1] > F2.shape[1])
    if swap:
        F1, F2 = F2, F1 # make the shorter sequence the query
        
    
    if F2.shape[1] / F1.shape[1] >= 2: # no valid path possible
        if max(F1.shape[1], F2.shape[1]) / min(F1.shape[1], F2.shape[1]) >= 2: # no valid path possible
            if outfile:
                pickle.dump(None, open(outfile, 'wb'))
            return None
    
    # start time logging
    times = []
    times.append(time.time())
    
    # Compute Cost
    C = 1 - F1.T @ F2
    
    times.append(time.time())
        
    # Subsequence DTW each segment without backtracing
    segLength = int(np.ceil(F1.shape[1] / segments))
    Cseg = np.zeros((segments, C.shape[1]), dtype = np.float64)
    Tseg = np.zeros((segments, F2.shape[1]), dtype=np.int32)
    Dparts = []
    Bparts = []
    
    for i in range(segments):
        
        segStart = i * segLength
        segEnd = min(segStart + segLength, C.shape[0])
        # Ensuring that the segment is contiguous here ensures best performance in later computations
        currentSeg = np.ascontiguousarray(C[segStart:segEnd,:])
        D_i, B_i = DTW_Cost_To_DandB(currentSeg, Steps = Steps, weights = weights, subsequence = True)
        
        # Store D_i and B_i for segment and construct Cseg
        Dparts.append(D_i)
        Bparts.append(B_i)
        
    
    times.append(time.time())
    
    for i in range(segments):
        D_i = Dparts[i]
        B_i = Bparts[i]
        Cseg[i,:] = D_i[-1,:]
        Tseg[i,:] = calcTseg(D_i, B_i, Steps=Steps)
    
    times.append(time.time())
    # run segment-level DTW (Not subsequence)
    Dseg, Bseg = SSDTW_Segment_Level_DTW(Cseg, Tseg)
    
    times.append(time.time())
        
    segmentEndIdxs = SSDTW_Segment_Level_Backtrace(Dseg, Bseg)
    
    times.append(time.time())
    
    # Frame level backtrace segment by segment
    path = []
    for i, endidx in enumerate(segmentEndIdxs):
        pathSeg = DTW_Backtrace(Dparts[i], Bparts[i], Steps = Steps, subsequence = True, startCol = endidx)
        
        # Add offset to row indices so they match with overall path
        pathSeg[0,:] = pathSeg[0,:] + (i * segLength)

        # Append fragment to full path
        path.append(pathSeg.copy())
    
    wp_merged = np.hstack(path)
    
    times.append(time.time())
    
    # Again, this swap is based on the old implementation
    if swap:
        wp_merged = np.flipud(wp_merged) # undo swap
        
    if outfile:
        pickle.dump(wp_merged, open(outfile, 'wb'))
    
    if profile:
        return wp_merged, np.diff(times)
    else:
        return wp_merged

In [ ]:
query_list = 'cfg_files/query.test.list'
featdir1 = FEATURES_ROOT / 'clean'
featdir2 = FEATURES_ROOT / 'clean' # in case you want to align clean vs noisy
n_cores = 1
steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
weights = np.array([1,1,2])
segmentVals = [2, 4, 8, 16, 32]
for numSegments in segmentVals:
    outdir = OUT_ROOT / f'experiments_test/clean/SSDTW_{numSegments}'
    SDTW_batch(query_list, featdir1, featdir2, outdir, n_cores, steps, weights, numSegments, SSDTW)